In [8]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np
from IPython.display import display




NA_VALUES = ["---", "--", "", " ", "NA", "N/A"]
# =========================
# FILE FINDER
# =========================
def find_file(name, start=Path.cwd()):
    for ancestor in [start] + list(start.parents):
        matches = list(ancestor.rglob(name))
        if matches:
            return matches[0]
    return None


def find_files(file_map):
    found = {}
    for key, filename in file_map.items():
        path = find_file(filename)
        if path:
            found[key] = path
        else:
            print(f"[WARNING] File not found: {filename}")
    return found

eda_script_path = find_file("script_eda.py")
if eda_script_path is None:
    raise FileNotFoundError("❌ script_eda.py tidak ditemukan di parent directory")

# tambahkan BASE PROJECT ke sys.path
sys.path.append(str(eda_script_path.parent))

# sekarang bisa import
from script_eda import evaluate_dataset, extract_column_schema,find_internal_duplicate_columns,extract_single_schema,cek_value_data_column








In [9]:
path = find_file("merged_cuaca_ndvi_ispu.csv")

if path is None:
    raise FileNotFoundError("❌ File merged tidak ditemukan")

df = pd.read_csv(path, na_values=NA_VALUES)

df.head()

,tanggal,periode_data,stasiun,pm_sepuluh,sulfur_dioksida,karbon_monoksida,ozon,nitrogen_dioksida,max,parameter_pencemar_kritis,kategori,id,time,temperature_2m_max (°C),temperature_2m_min (°C),precipitation_sum (mm),precipitation_hours (h),wind_speed_10m_max (km/h),wind_direction_10m_dominant (°),shortwave_radiation_sum (MJ/m²),temperature_2m_mean (°C),relative_humidity_2m_mean (%),cloud_cover_mean (%),surface_pressure_mean (hPa),wind_gusts_10m_max (km/h),winddirection_10m_dominant (°),relative_humidity_2m_max (%),relative_humidity_2m_min (%),cloud_cover_max (%),cloud_cover_min (%),wind_gusts_10m_mean (km/h),wind_speed_10m_mean (km/h),wind_gusts_10m_min (km/h),wind_speed_10m_min (km/h),surface_pressure_max (hPa),surface_pressure_min (hPa),lokasi,lokasi_clean,ndvi
0,2010-01-01,201001,DKI1 (Bunderan HI),60.0,4.0,73.0,27.0,14.0,73.0,CO,SEDANG,2010-01-01_DKI1,2010-01-01,29.4,24.4,4.0,14.0,16.0,246.0,16.24,26.6,81.0,100.0,1007.5,38.2,246.0,90.0,69.0,100.0,99.0,21.0,10.5,11.9,6.9,1009.3,1005.1,dki1_bundaranhi,DKI1,0.2023
1,2010-01-02,201001,DKI1 (Bunderan HI),32.0,2.0,16.0,33.0,9.0,33.0,O3,BAIK,2010-01-02_DKI1,2010-01-02,30.8,24.0,6.5,4.0,14.7,238.0,19.80,26.9,82.0,99.0,1008.1,31.3,238.0,93.0,69.0,100.0,91.0,16.5,7.7,9.0,4.4,1009.9,1006.0,dki1_bundaranhi,DKI1,0.2023
2,2010-01-03,201001,DKI1 (Bunderan HI),27.0,2.0,19.0,20.0,9.0,27.0,PM10,BAIK,2010-01-03_DKI1,2010-01-03,30.4,24.3,7.6,11.0,12.6,244.0,17.32,26.7,83.0,97.0,1008.5,26.6,244.0,91.0,70.0,100.0,81.0,18.4,9.4,11.9,6.5,1010.5,1006.5,dki1_bundaranhi,DKI1,0.2023
3,2010-01-04,201001,DKI1 (Bunderan HI),22.0,2.0,16.0,15.0,6.0,22.0,PM10,BAIK,2010-01-04_DKI1,2010-01-04,30.3,25.4,0.9,5.0,19.3,239.0,20.43,27.4,81.0,69.0,1007.5,34.2,239.0,91.0,70.0,100.0,17.0,23.8,13.5,14.4,9.6,1009.1,1005.1,dki1_bundaranhi,DKI1,0.2023
4,2010-01-05,201001,DKI1 (Bunderan HI),25.0,2.0,17.0,15.0,8.0,25.0,PM10,BAIK,2010-01-05_DKI1,2010-01-05,29.9,24.8,14.3,7.0,15.9,247.0,15.86,26.4,83.0,100.0,1007.7,32.0,247.0,89.0,72.0,100.0,99.0,21.6,11.1,10.4,7.8,1009.1,1006.0,dki1_bundaranhi,DKI1,0.2023


In [10]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# load data
path = find_file("merged_cuaca_ndvi_ispu.csv")
if path is None:
    raise FileNotFoundError("❌ File merged tidak ditemukan")

df = pd.read_csv(path, na_values=NA_VALUES)

# ensure consistent types
df["periode_data"] = df["periode_data"].astype(str)
df["lokasi_clean"] = df["lokasi_clean"].astype(str)

# aggregate per periode
periode_summary = (
    df.groupby("periode_data")
      .agg(
          jumlah_row=("periode_data", "size"),
          lokasi_list=("lokasi_clean", lambda x: sorted(x.unique().tolist()))
      )
      .reset_index()
      .sort_values("periode_data")
)

periode_summary




,periode_data,jumlah_row,lokasi_list
0,201001,31,[DKI1]
1,201002,28,[DKI1]
2,201003,31,[DKI1]
3,201004,30,[DKI1]
4,201005,31,[DKI1]
5,201006,29,[DKI1]
6,201007,31,[DKI1]
7,201008,30,[DKI1]
8,201009,30,[DKI1]
9,201010,31,[DKI1]


In [11]:
# expected rows per month (5 stasiun × ~30 hari)
EXPECTED_MIN = 130
EXPECTED_MAX = 160

periode_counts["status"] = periode_counts["jumlah_row"].apply(
    lambda x: "OK" if EXPECTED_MIN <= x <= EXPECTED_MAX else "INCOMPLETE"
)

periode_counts


,periode_data,jumlah_row,status
0,201001,31,INCOMPLETE
1,201002,28,INCOMPLETE
2,201003,31,INCOMPLETE
3,201004,30,INCOMPLETE
4,201005,31,INCOMPLETE
5,201006,29,INCOMPLETE
6,201007,31,INCOMPLETE
7,201008,30,INCOMPLETE
8,201009,30,INCOMPLETE
9,201010,31,INCOMPLETE


In [12]:
# =========================
# 2. BASIC CLEANING
# =========================
# parse date
df["tanggal"] = pd.to_datetime(
    df["tanggal"],
    dayfirst=True,
    errors="coerce"
)

# ensure correct types
df["lokasi_clean"] = df["lokasi_clean"].astype(str)
df["kategori"] = df["kategori"].astype(str)
df["periode_data"] = df["periode_data"].astype(str)

# drop rows missing essential fields
df = df.dropna(subset=["tanggal", "lokasi_clean", "kategori"])

# =========================
# 3. SORT (CRITICAL)
# =========================
df = df.sort_values(
    ["lokasi_clean", "tanggal"]
).reset_index(drop=True)

# =========================
# 4. CREATE NEXT-DAY COLUMNS (PER STATION)
# =========================
df["tanggal_next"] = df.groupby("lokasi_clean")["tanggal"].shift(-1)
df["kategori_next"] = df.groupby("lokasi_clean")["kategori"].shift(-1)

# =========================
# 5. DETECT VALID DAY-TO-DAY TRANSITIONS
# =========================
df["is_valid_transition"] = (
    (df["tanggal_next"] - df["tanggal"]) == pd.Timedelta(days=1)
)

# =========================
# 6. KEEP ONLY VALID TRANSITIONS
# =========================
transitions = df[df["is_valid_transition"]].copy()

transitions = transitions[
    [
        "lokasi_clean",
        "tanggal",
        "kategori",
        "tanggal_next",
        "kategori_next",
        "periode_data"
    ]
]

# =========================
# 7. SANITY CHECKS
# =========================

# a) number of valid transitions per station
transition_counts = (
    transitions.groupby("lokasi_clean")
               .size()
               .reset_index(name="jumlah_transisi_valid")
)

print("Jumlah transisi valid per stasiun:")
print(transition_counts)

# b) optional: inspect broken sequences (gaps)
broken_sequences = df[
    (~df["is_valid_transition"]) & (df["tanggal_next"].notna())
][["lokasi_clean", "tanggal", "tanggal_next"]]

print("\nContoh gap (non-konsekutif):")
print(broken_sequences.head(10))

# =========================
# 8. OPTIONAL: SAVE OUTPUT
# =========================

Jumlah transisi valid per stasiun:
  lokasi_clean  jumlah_transisi_valid
0         DKI1                    990
1         DKI2                    955
2         DKI3                    869
3         DKI4                   1008
4         DKI5                    934

Contoh gap (non-konsekutif):
   lokasi_clean    tanggal tanggal_next
9          DKI1 2010-01-10   2010-01-12
10         DKI1 2010-01-12   2010-02-01
22         DKI1 2010-02-12   2010-03-01
34         DKI1 2010-03-12   2010-04-01
41         DKI1 2010-04-07   2010-04-09
45         DKI1 2010-04-12   2010-05-01
57         DKI1 2010-05-12   2010-06-01
69         DKI1 2010-06-12   2010-07-01
81         DKI1 2010-07-12   2010-08-01
93         DKI1 2010-08-12   2010-09-01
